In [1]:
import copy
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from lsst.sims.catUtils.baseCatalogModels import BaseCatalogModels as bcm
from lsst.sims.catalogs.generation.db import CatalogDBObject, ChunkIterator
from lsst.sims.catalogs.measures.instance import InstanceCatalog
from lsst.sims.catalogs.generation.db.ObservationMetaData import ObservationMetaData

/usr/local/manual/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want

In [29]:
TryOn = True
while TryOn:
    try: 
        cat = CatalogDBObject.from_objid('galaxyTiled')
        TryOn = False
        print 'Connected'
    except:
        print "Failed to create Connection on fatboy, Trying Again"
    

Connected


In [19]:
mycolnames = ['htmid', 'galtileid', 'id', 'redshift', 'mass_stellar', 'raJ2000', 'decJ2000']

In [5]:
Kvals  = np.loadtxt('numSNperMass.dat')
Kvals *=  16.0 / (np.pi* (1.75)**2.)
print type(Kvals)
print Kvals 

<type 'numpy.ndarray'>
[ 0.03600648  0.02044498  0.02380302  0.02842369  0.02509181  0.02640672
  0.02726142  0.03133616  0.0272369   0.02776356  0.02480492  0.02610207]


In [6]:
def fieldCenters(fieldID):
    deepFields = pd.DataFrame.from_csv('data/DeepFields.dat', index_col=0)
    coords = deepFields[deepFields['fieldID']==fieldID][['fieldRA', 'fieldDec']].values
    return coords

In [7]:
cc = fieldCenters(744)

In [8]:
cc[0,0]

0.0

In [9]:
cc[0,1]

-45.524533499458101

In [10]:
def writeToDisk(fname, chunk, writes, colnames, kval, masktype='SN'):
    #print colnames
    print len(chunk)
    df = pd.DataFrame(np.asarray(map(list, chunk)), columns=mycolnames)
    df[['htmid', 'galtileid', 'id']] = df[['htmid', 'galtileid', 'id']].astype(int)
    numRecs = len(df)
    r = np.random.uniform(0., 1., size=numRecs)
    if masktype == 'SN':
        print 'Doing SN mask'
        print kval, df.mass_stellar.max(), kval * df.mass_stellar.max() 
        mask = r / df.mass_stellar.values / kval  < 1.0
    else:
        mask = r < float(masktype)
    if any(mask):
        if 0 == writes:
            df[mask].to_csv(fname, mode='w', index=False)
        else:
            df[mask].to_csv(fname, mode='a', index=False, header=False)
        return len(df[mask])
    return 0

In [26]:
fieldCenters(1427)

array([[ 53.00913847, -27.43894881]])

In [11]:
def getgalids(fname, fieldID, redshiftbin, kval, extraconst='',
              masktype='SN', returnNone=True, chunkcount=0):
    
    redshifts = np.linspace(0., 1.2, 13)
    redshiftlow = redshifts[redshiftbin]
    redshifthigh = redshifts[redshiftbin+1]
    fieldCoords = fieldCenters(fieldID)
    colnames = copy.deepcopy(mycolnames)
    constraintStr = 'redshift < ' + "{0:.2f}".format(redshifthigh) \
                  + ' and redshift > ' + "{1:.2f}".format(redshifthigh, redshiftlow)
    constraintStr += extraconst
    print constraintStr
    myObsMD = ObservationMetaData(boundType='circle',
                                  boundLength=1.75,
                                  unrefractedRA=fieldCoords[0, 0],
                                  unrefractedDec=fieldCoords[0, 1])
    catalogIterator = cat.query_columns(colnames=colnames, 
                                        chunk_size=20000, 
                                        obs_metadata=myObsMD, 
                                        constraint=constraintStr)
    chunkNum = 0
    for chunk in catalogIterator:
        # print 'chunkNum ', chunkNum
        chunkNum += 1
        chunkcount += writeToDisk(fname, chunk, writes=chunkcount, 
                                  colnames=mycolnames, kval=kval,
                                  masktype=masktype)
    numGals = (chunkNum - 1)* catalogIterator.chunk_size + chunk.size
    print 'Num Gals '+ "{0:6d}".format(int(numGals))
    print 'Num SN' + "{0:6d}".format(int(chunkcount))
    if not returnNone:
        return pd.DataFrame.from_csv(fname)

In [12]:
extraconst_str = ' and mass_stellar > 0.01' 
np.random.seed(42)
for redshiftbin, kval in enumerate(Kvals):
   getgalids('galIds_full_290.csv', 290, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

redshift < 0.10 and redshift > 0.00 and mass_stellar > 0.01
2596
Doing SN mask
0.036006478021 14.0909986496 0.507367233171
Num Gals   2596
Num SN    24
redshift < 0.20 and redshift > 0.10 and mass_stellar > 0.01
20000
Doing SN mask
0.0204449786702 36.5867462158 0.748015245996
9329
Doing SN mask
0.0204449786702 31.7024860382 0.648156650845
Num Gals  29329
Num SN   312
redshift < 0.30 and redshift > 0.20 and mass_stellar > 0.01
20000
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
20000
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
10387
Doing SN mask
0.0238030166713 49.4887619019 1.17798182459
Num Gals  50387
Num SN   437
redshift < 0.40 and redshift > 0.30 and mass_stellar > 0.01
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.976348877 0.965733102253
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.02842368689

In [13]:
np.random.seed(42)

In [14]:
extraconst_str = ' and mass_stellar > 0.01' 
for redshiftbin, kval in enumerate(Kvals):
   getgalids('galIds_290.csv', 290, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

redshift < 0.10 and redshift > 0.00 and mass_stellar > 0.01
2596
Doing SN mask
0.036006478021 14.0909986496 0.507367233171
Num Gals   2596
Num SN    28
redshift < 0.20 and redshift > 0.10 and mass_stellar > 0.01
20000
Doing SN mask
0.0204449786702 36.5867462158 0.748015245996
9329
Doing SN mask
0.0204449786702 31.7024860382 0.648156650845
Num Gals  29329
Num SN   297
redshift < 0.30 and redshift > 0.20 and mass_stellar > 0.01
20000
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
20000
Doing SN mask
0.0238030166713 49.4887619019 1.17798182459
10387
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
Num Gals  50387
Num SN   477
redshift < 0.40 and redshift > 0.30 and mass_stellar > 0.01
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.976348877 0.965733102253
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.02842368689

In [15]:
np.random.seed(42)

In [16]:
extraconst_str = ' and mass_stellar > 0.01' 
for redshiftbin, kval in enumerate(Kvals):
   getgalids('galIds_1427.csv', 1427, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

redshift < 0.10 and redshift > 0.00 and mass_stellar > 0.01
2736
Doing SN mask
0.036006478021 18.7934265137 0.676685098704
Num Gals   2736
Num SN    31
redshift < 0.20 and redshift > 0.10 and mass_stellar > 0.01
20000
Doing SN mask
0.0204449786702 36.5867462158 0.748015245996
4657
Doing SN mask
0.0204449786702 36.5867462158 0.748015245996
Num Gals  24657
Num SN   261
redshift < 0.30 and redshift > 0.20 and mass_stellar > 0.01
20000
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
20000
Doing SN mask
0.0238030166713 49.4887619019 1.17798182459
10597
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
Num Gals  50597
Num SN   513
redshift < 0.40 and redshift > 0.30 and mass_stellar > 0.01
20000
Doing SN mask
0.0284236868932 33.0772285461 0.94017678749
20000
Doing SN mask
0.0284236868932 33.976348877 0.965733102253
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.02842368689

In [17]:
np.random.seed(42)
extraconst_str = ' and mass_stellar > 0.01' 
for redshiftbin, kval in enumerate(Kvals):
   getgalids('galIds_2786.csv', 2786, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

redshift < 0.10 and redshift > 0.00 and mass_stellar > 0.01
2364
Doing SN mask
0.036006478021 14.6567602158 0.527738314568
Num Gals   2364
Num SN    20
redshift < 0.20 and redshift > 0.10 and mass_stellar > 0.01
20000
Doing SN mask
0.0204449786702 36.5867462158 0.748015245996
462
Doing SN mask
0.0204449786702 14.0149869919 0.286536110113
Num Gals  20462
Num SN   216
redshift < 0.30 and redshift > 0.20 and mass_stellar > 0.01
20000
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
20000
Doing SN mask
0.0238030166713 49.4887619019 1.17798182459
19670
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
Num Gals  59670
Num SN   579
redshift < 0.40 and redshift > 0.30 and mass_stellar > 0.01
20000
Doing SN mask
0.0284236868932 33.976348877 0.965733102253
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.028423686893

In [24]:
np.random.seed(42)
extraconst_str = ' and mass_stellar > 0.01' 
for redshiftbin, kval in enumerate(Kvals):
   getgalids('galIds_2412.csv', 2412, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

redshift < 0.10 and redshift > 0.00 and mass_stellar > 0.01
2378
Doing SN mask
0.036006478021 14.6567602158 0.527738314568
Num Gals   2378
Num SN    20
redshift < 0.20 and redshift > 0.10 and mass_stellar > 0.01
20000
Doing SN mask
0.0204449786702 36.5867462158 0.748015245996
393
Doing SN mask
0.0204449786702 11.0948619843 0.226834216617
Num Gals  20393
Num SN   234
redshift < 0.30 and redshift > 0.20 and mass_stellar > 0.01
20000
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
20000
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
19423
Doing SN mask
0.0238030166713 49.4887619019 1.17798182459
Num Gals  59423
Num SN   594
redshift < 0.40 and redshift > 0.30 and mass_stellar > 0.01
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.028423686893

In [30]:
np.random.seed(42)
extraconst_str = ' and mass_stellar > 0.01' 
for redshiftbin, kval in enumerate(Kvals):
   getgalids('galIds_744.csv', 744, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

redshift < 0.10 and redshift > 0.00 and mass_stellar > 0.01
2395
Doing SN mask
0.036006478021 15.3677091599 0.553337082097
Num Gals   2395
Num SN    24
redshift < 0.20 and redshift > 0.10 and mass_stellar > 0.01
20000
Doing SN mask
0.0204449786702 36.5867462158 0.748015245996
2131
Doing SN mask
0.0204449786702 18.1802711487 0.371695255854
Num Gals  22131
Num SN   227
redshift < 0.30 and redshift > 0.20 and mass_stellar > 0.01
20000
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
20000
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
19426
Doing SN mask
0.0238030166713 65.7484130859 1.56501057279
Num Gals  59426
Num SN   567
redshift < 0.40 and redshift > 0.30 and mass_stellar > 0.01
20000
Doing SN mask
0.0284236868932 33.7322425842 0.95879470142
20000
Doing SN mask
0.0284236868932 33.976348877 0.965733102253
20000
Doing SN mask
0.0284236868932 33.976348877 0.965733102253
20000
Doing SN mask
0.0284236868932 33.976348877 0.965733102253
20000
Doing SN mask
0.02842368689

# Scratch

In [ ]:
colnames = copy.deepcopy(mycolnames)
catalogIterator = cat.query_columns(colnames=colnames, chunk_size=20000, obs_metadata=myObsMD, 
                                    constraint='redshift < 1.2 and redshift > 1.1')

In [ ]:
colnames

In [ ]:
Kvals  = np.loadtxt('numSNperMass.dat')

In [ ]:
deepFields = pd.DataFrame.from_csv('data/DeepFields.dat', index_col=0)

In [ ]:
deepFields

In [ ]:
df1 = pd.DataFrame.from_csv('galIds_1427.csv')
df2 = pd.DataFrame.from_csv('galIds_2412.csv')

In [ ]:
plt.hist(df1.mass_stellar.apply(lambda x: np.log10(x)), bins=np.linspace(0.01, 30, 60), histtype='step')
plt.hist(df2.mass_stellar.apply(lambda x: np.log10(x)), bins=np.linspace(0.01, 30, 60), histtype='step')

In [ ]:
plt.hist(df1.redshift.values, bins=np.linspace(0., 1.2, 13), histtype='step')
plt.hist(df2.redshift.values, bins=np.linspace(0., 1.2, 13), histtype='step')

In [ ]:
one = df1.id.values
two = df2.id.values

In [ ]:
o = np.sort(one)
t = np.sort(two)

In [ ]:
o[:5]

In [ ]:
t[:5]

In [ ]:
fieldCenters(744)

In [ ]:
def getgalids(fname, fieldID, redshiftlow, redshifthigh, extraconst='',
              masktype='SN', returnNone=True):
    
    fieldCoords = fieldCenters(fieldID)
    colnames = copy.deepcopy(mycolnames)
    constraintStr = 'redshift <' \
                  + "{0:.2f}" \
                  + 'and redshift > ' + "{1:.2f}".format(redshifthigh, redshiftlow)
    myObsMD = ObservationMetaData(boundType='circle',
                                  boundLength=1.75,
                                  unrefractedRA=53.01,
                                  unrefractedDec=-27.43)
    catalogIterator = cat.query_columns(colnames=colnames, 
                                        chunk_size=20000, 
                                        obs_metadata=myObsMD, 
                                        constraint=constraintStr \
                                        + extraconst)
    chunkNum = 0
    chunkcount = 0
    for chunk in catalogIterator:
        print 'chunkNum ', chunkNum
        chunkNum += 1
        chunkcount += writeToDisk(fname, chunk, writes=chunkcount, colnames=mycolnames, kval=Kvals[2],
                              masktype=masktype)
    numGals = (chunkNum - 1)* catalogIterator.chunk_size + chunk.size
    print 'Num Gals '+ "{0:6d}".format(int(numGals))
    if not returnNone:
        return pd.DataFrame.from_csv(fname)

In [ ]:
counts = []
data = []
chunkcount = 0
np.random.seed(42)
chunkNum = 0
for chunk in catalogIterator:
    #print colnames
    #data=np.asarray(map(list, chunk))
    print 'chunkNum ', chunkNum
    chunkNum += 1
    chunkcount += writeToDisk('SNgalids.csv', chunk, writes=chunkcount, colnames=mycolnames, kval=Kvals[2],
                              )

In [ ]:
chunk.size

In [ ]:
colnames = copy.deepcopy(mycolnames)
catalogIterator = cat.query_columns(colnames=colnames, chunk_size=20000, obs_metadata=myObsMD, 
                                    constraint='redshift < 1.2 and redshift > 1.1')

In [ ]:
counts = []
data = []
chunkcount = 0
np.random.seed(42)
chunkNum = 0
for chunk in catalogIterator:
    #print colnames
    #data=np.asarray(map(list, chunk))
    print 'chunkNum ', chunkNum
    chunkNum += 1
    chunkcount += writeToDisk('Galgalids.csv', chunk, writes=chunkcount, colnames=mycolnames, kval=Kvals[2],
                              masktype='0.01')

# Time Grid

In [ ]:
def writeToDiskFull(fname, chunk, writes, colnames, kval, masktype='SN'):
    #print colnames
    print len(chunk)
    df = pd.DataFrame(np.asarray(map(list, chunk)), columns=mycolnames)
    df[['htmid', 'galtileid', 'id']] = df[['htmid', 'galtileid', 'id']].astype(int)
    numRecs = len(df)
    r = np.random.uniform(-0.5, 0.5, size=numRecs)
    print 'Doing SN mask'
    print kval, df.mass_stellar.max(), kval * df.mass_stellar.max() 
    df['t'] =  r * 0.5 / df.mass_stellar.values / kval
    if 0 == writes:
        df.to_csv(fname, mode='w', index=False)
    else:
        df.to_csv(fname, mode='a', index=False, header=False)
    return len(df)
   

In [ ]:
def putTimes(fname, fieldID, redshiftbin, kval, extraconst='',
              masktype='SN', returnNone=True, chunkcount=0):
    
    redshifts = np.linspace(0., 1.2, 13)
    redshiftlow = redshifts[redshiftbin]
    redshifthigh = redshifts[redshiftbin+1]
    fieldCoords = fieldCenters(fieldID)
    colnames = copy.deepcopy(mycolnames)
    constraintStr = 'redshift < ' + "{0:.2f}".format(redshifthigh) \
                  + ' and redshift > ' + "{1:.2f}".format(redshifthigh, redshiftlow)
    constraintStr += extraconst
    print constraintStr
    myObsMD = ObservationMetaData(boundType='circle',
                                  boundLength=1.75,
                                  unrefractedRA=53.01,
                                  unrefractedDec=-27.43)
    catalogIterator = cat.query_columns(colnames=colnames, 
                                        chunk_size=20000, 
                                        obs_metadata=myObsMD, 
                                        constraint=constraintStr)
    chunkNum = 0
    for chunk in catalogIterator:
        # print 'chunkNum ', chunkNum
        chunkNum += 1
        chunkcount += writeToDisk(fname, chunk, writes=chunkcount, 
                                  colnames=mycolnames, kval=kval,
                                  masktype=masktype)
    numGals = (chunkNum - 1)* catalogIterator.chunk_size + chunk.size
    print 'Num Gals '+ "{0:6d}".format(int(numGals))
    print 'Num SN' + "{0:6d}".format(int(chunkcount))
    if not returnNone:
        return pd.DataFrame.from_csv(fname)

In [ ]:
extraconst_str = ' and mass_stellar > 0.01'
np.random.seed(42)
for redshiftbin, kval in enumerate(Kvals):
   putTimes('galIds_full_744.csv', 744, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

In [ ]:
extraconst_str = ' and mass_stellar > 0.01'
np.random.seed(42)
for redshiftbin, kval in enumerate(Kvals):
   putTimes('galIds_full_290.csv', 290, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

In [ ]:
extraconst_str = ' and mass_stellar > 0.01'
np.random.seed(42)
for redshiftbin, kval in enumerate(Kvals):
   putTimes('galIds_full_1427.csv', 1427, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

In [ ]:
extraconst_str = ' and mass_stellar > 0.01'
np.random.seed(42)
for redshiftbin, kval in enumerate(Kvals):
   putTimes('galIds_full_2412.csv', 2412, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

In [ ]:
extraconst_str = ' and mass_stellar > 0.01'
np.random.seed(42)
for redshiftbin, kval in enumerate(Kvals):
   putTimes('galIds_full_2786.csv', 2786, redshiftbin, kval=kval, 
             extraconst=extraconst_str,
              masktype='SN', returnNone=True, chunkcount=redshiftbin)

In [ ]:
# df = pd.DataFrame.from_csv('galids.csv')

In [ ]:
SNdf = pd.DataFrame.from_csv('SNgalids.csv')

In [ ]:
Galdf = pd.DataFrame.from_csv('Galgalids.csv')

In [ ]:
len(Galdf)

In [ ]:
len(SNdf)

In [ ]:
bins = [ -2.30560542e+00,  -1.92096501e+00,  -1.53632459e+00,
         -1.15168417e+00,  -7.67043757e-01,  -3.82403341e-01,
          2.23707537e-03,   3.86877492e-01,   7.71517908e-01,
          1.15615832e+00,   1.54079874e+00]

In [ ]:
plt.hist(SNdf['mass_stellar'].apply(lambda x: np.log10(x)).values, histtype='step',
         bins=bins, color='r', label='SN')
plt.hist(galdf['mass_stellar'].apply(lambda x: np.log10(x)).values, histtype='step',
         bins=bins, color='b', label='gals')
plt.legend(loc='best')

In [ ]:
df.dtypes

In [ ]:
colnames

In [ ]:
help(pd.DataFrame)

In [ ]:
counts

In [ ]:
len(chunk)

In [ ]:
x = np.asarray(map(list, chunk))

Note
====
- colnames: Must include raJ2000, and decJ2000, or there will be a crash
- chunksize in the docstrings is actually chunk_size

In [ ]:
help(cat.query_columns)

In [ ]:
cat = CatalogDBObject.from_objid('galaxyBase')

In [ ]:
randomSeed = 42

In [ ]:
np.random.seed(randomSeed)

In [ ]:
DeepFieldsDF = pd.DataFrame.from_csv('../data/DeepFields.dat')

In [ ]:
DeepFieldsDF

In [ ]:
catalogIterator = cat.query_columns(colnames=['galid', 'redshift', 'mass_stellar'], 
                                    #chunksize = 10000,
                                    constraint='mass_stellar > 0.01 and redshift > 0. and redshift < 0.1')

In [ ]:
zlow =0.


In [ ]:
constraint = 'mass_stellar > 0.01 and redshift > 0.0 and redshift < 0.1 and ra > 51 and ra < 55 and dec > -29.5 and dec < -25.5' 

In [ ]:
catalogIterator = cat.query_columns(colnames=['galaxyTiledID', 'redshift', 'mass_stellar'], constraint=constraint, chunk_size=10000)

In [ ]:
catalogIterator = cat.query_columns(colnames=['htmid', 'galtileid', 'redshift', 'mass_stellar'], 
                                    chunk_size = 10000,
                                    constraint='mass_stellar > 0.01 and redshift < 0.05')

In [ ]:
catalogIterator1 = cat.query_columns(colnames=['redshift', 'mass_stellar'],constraint = 'redshift between 0.5 and 0.6')

In [ ]:
cat.query_columns()

In [ ]:
myObsMD = ObservationMetaData(boundType='circle',
                                      boundLength=1.75,
                                      unrefractedRA=53.01,
                                      unrefractedDec=-27.43)